# Query S2 Samples

Here I am trying to find all S2 samples and possibly do a little cleanup.

In [1]:
# %load ../config/defaults.py
# Load useful extensions

# Activate the autoreload extension for easy reloading of external packages
%reload_ext autoreload
%autoreload 2

# Trun on the water mark
%reload_ext watermark
%watermark -u -d -g

# Load ipycache extension
%reload_ext ipycache
from ipycache import CacheMagics
CacheMagics.cachedir = '../cachedir'

# Add project library to path
import sys
sys.path.insert(0, '../../lib/python')

# The usual suspects
import os
import numpy as np
import pandas as pd

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_context('poster')

# Turn off scientific notation
np.set_printoptions(precision=5, suppress=True)


last updated: 2017-04-24 
Git hash: 46dde429dab4bedbca6fc69b24c670a949be87af


In [2]:
import re
from pymongo import MongoClient

In [3]:
client = MongoClient(port=27022)
db = client['test']
biometa = db['biometa']

In [4]:
regex = re.compile(r'.*s2.*', re.IGNORECASE)
agg = list(biometa.aggregate([
    {
        '$match': {'magic': {'$exists': 1}}
    },
    {
        '$unwind': '$magic'
    },
    {
        '$match': {
            'magic.name': 'sample summary',
            'magic.value': {'$regex': regex}
        }
    },
    {
        '$unwind': '$experiments'
    },
    {
        '$unwind': '$experiments.runs'
    },
    {
        '$project': {'_id': 0, 'biosample': '$_id', 'srx': '$experiments.srx', 'srr': '$experiments.runs', 'magic summary': '$magic.value'}
    }
]))

In [5]:
s2SRR = [x['srr'] for x in agg]

In [6]:
with open('../../data/13495_runs_analyzed_by_mieg.txt') as fh:
    mieg = [x.strip() for x in fh.readlines()]

In [7]:
okS2 = [x for x in s2SRR if x in mieg]

In [8]:
with open('../../data/1104_s2_cell_in_mieg.txt', 'w') as fh:
    for s2 in okS2:
        fh.write(s2 + '\n')

In [10]:
s2SRR = [x['biosample'] for x in agg]